In [1]:
# connect to the server

from pprint import pprint
import json

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [2]:
with open("private.json", "r") as f:
    private = json.load(f)

uri = private["MONGO_ATLAS_URI"]

# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi("1"))

# Send a ping to confirm a successful connection
try:
    client.admin.command("ping")
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [3]:
ohio_db = client.ohio_db
cities_collection = ohio_db.cities
boundaries_collection = ohio_db.boundaries

boundaries_collection.create_index([("geometry", "2dsphere")])
cities_collection.create_index([("geometry", "2dsphere")])

'geometry_2dsphere'

In [4]:
franklin_boundary = boundaries_collection.find_one(
    {"properties.name": "Franklin County"}
)
if not franklin_boundary:
    print("Franklin County boundary not found in the database.")
    exit()
    
cities_within_franklin = cities_collection.find(
    {"geometry": {"$geoWithin": {"$geometry": franklin_boundary["geometry"]}}}
)

print("Cities within Franklin County:")
for city in cities_within_franklin:
    print(city["properties"]["City"])

Cities within Franklin County:
Columbus


In [6]:
def cities_within_county(county_name):
    boundary = boundaries_collection.find_one(
        {"properties.name": county_name}
    )
    if not boundary:
        print(f"{county_name} boundary not found in the database.")
        return

    cities = cities_collection.find(
            {"geometry": {"$geoWithin": {"$geometry": boundary["geometry"]}}}   
        )

    print(f"Cities within {county_name}:")
    for city in cities:
        print(city["properties"]["City"])

cities_within_county("Cuyahoga County")

Cities within Cuyahoga County:
Parma
Cleveland
Lakewood
